In [15]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
  .master("local[*]") \
  .appName('1.2. BigQuery Storage & Spark SQL - Python')\
  .config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest.jar') \
  .getOrCreate()

In [2]:
table = "applied-mystery-341809.dbt_week5.dim_zones"
df_wiki_pageviews = spark.read \
  .format("bigquery") \
  .option("table", table) \
  .load()

In [3]:
df_wiki_pageviews.show()

+----------+-------+--------------------+------------+
|locationid|borough|                zone|service_zone|
+----------+-------+--------------------+------------+
|         1|   NEWR|      Newark Airport|         EWR|
|       265|Unknown|                  NA|         N/A|
|       264|Unknown|                  NV|         N/A|
|       138| Queens|   LaGuardia Airport|    Airports|
|       132| Queens|         JFK Airport|    Airports|
|       168|  Bronx|Mott Haven/Port M...|  Green Zone|
|       208|  Bronx|Schuylerville/Edg...|  Green Zone|
|         3|  Bronx|Allerton/Pelham G...|  Green Zone|
|        58|  Bronx|        Country Club|  Green Zone|
|       212|  Bronx|  Soundview/Bruckner|  Green Zone|
|        59|  Bronx|        Crotona Park|  Green Zone|
|       220|  Bronx|Spuyten Duyvil/Ki...|  Green Zone|
|        81|  Bronx|         Eastchester|  Green Zone|
|        18|  Bronx|        Bedford Park|  Green Zone|
|        60|  Bronx|   Crotona Park East|  Green Zone|
|        7

In [11]:
df_artists = spark.read \
.option("header",True) \
.parquet("gs://dtc_data_lake_applied-mystery-341809/transform/artists")

In [13]:
df_artists.show(10)

+--------------------+---------+--------------------+
|                  id|followers|                name|
+--------------------+---------+--------------------+
|59qz10hjQPAs0spos...|     18.0|    Boys of the Band|
|0LcrfJ63GdIW4n2UZ...|      0.0|David Mackersie e...|
|18lEjlk2JBPdyObT3...|    785.0|Ministerio Doble ...|
|1UhZGGHbPPHAHt2ce...|    214.0|  Andre Tschaskowski|
|5D95DPWHohrzVbdub...|  10557.0|       Advent Sorrow|
|11CbG4ImkEw99aUng...|   3614.0|                  MX|
|61MH29rMIyOfuK7KX...|  27878.0| The Vintage Caravan|
|5lInFfKjIAVzBIOg4...|    445.0|     Charles Grigsby|
|3kJakFcxRwLW9f47x...|      4.0|        Willy Rustad|
|508weSx4HBumrGggF...|   1110.0|              Hunxho|
+--------------------+---------+--------------------+
only showing top 10 rows



In [16]:
# Update to your GCS bucket
gcs_bucket = 'dtc_data_lake_applied-mystery-341809'

# Update to your BigQuery dataset name you created
bq_dataset = 'bq_project_dezoomcamp'

# Enter BigQuery table name you want to create or overwite. 
# If the table does not exist it will be created when you run the write function
bq_table = 'dim_artists'

df_artists.write \
  .format("bigquery") \
  .option("table","{}.{}".format(bq_dataset, bq_table)) \
  .option("temporaryGcsBucket", gcs_bucket) \
  .mode('overwrite') \
  .save()